In [ ]:
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
class QuadState(TypedDict):
    a: int
    b: int
    c: int
    equation: str
    discriminant: float
    result: str

In [ ]:
def show_equation(state: QuadState):
    eq = f"{state['a']}x² + {state['b']}x + {state['c']} = 0"
    return {'equation': eq}

def calculate_discriminant(state: QuadState):
    d = state["b"]**2 - (4 * state["a"] * state["c"])
    return {'discriminant': d}

def real_roots(state: QuadState):
    r1 = (-state["b"] + state["discriminant"]**0.5) / (2 * state["a"])
    r2 = (-state["b"] - state["discriminant"]**0.5) / (2 * state["a"])
    return {'result': f'Roots are real and distinct: {r1} and {r2}'}

def repeated_roots(state: QuadState):
    r = -state["b"] / (2 * state["a"])
    return {'result': f'Root is real and repeated: {r}'}

def no_real_roots(state: QuadState):
    return {'result': 'The equation has no real roots (complex roots)'}

In [ ]:
def check_condition(state: QuadState) -> Literal["real_roots", "repeated_roots", "no_real_roots"]:
    if state['discriminant'] > 0:
        return "real_roots"
    elif state['discriminant'] == 0:
        return "repeated_roots"
    else:
        return "no_real_roots"

In [ ]:
graph = StateGraph(QuadState)

graph.add_node('show_equation', show_equation)
graph.add_node('calculate_discriminant', calculate_discriminant)
graph.add_node('real_roots', real_roots)
graph.add_node('repeated_roots', repeated_roots)
graph.add_node('no_real_roots', no_real_roots)

graph.add_edge(START, 'show_equation')
graph.add_edge('show_equation', 'calculate_discriminant')
graph.add_conditional_edges('calculate_discriminant', check_condition)

graph.add_edge('real_roots', END)
graph.add_edge('repeated_roots', END)
graph.add_edge('no_real_roots', END)

workflow = graph.compile()

In [ ]:
inputs = {'a': 1, 'b': -5, 'c': 6} 
result = workflow.invoke(inputs)
import pprint
pprint.pprint(result)